In [57]:
#treinamento do modelo

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import LabelEncoder


# Carregar os dados financeiros de candles a partir de um arquivo CSV
df = base

# Pre-processar os dados
# Eliminar valores ausentes
df.dropna(inplace=True)
# Codificação de categoria
encoder = LabelEncoder()
df.SYMBOL = encoder.fit_transform(df[['SYMBOL']])
# Converter datas em variáveis numéricas
df['DATE'] = pd.to_datetime(df['DATE'].apply(lambda x: datetime.datetime(x.year, x.month, x.day)))
df['DATE'] = df['DATE'].astype('int64') / 10**9
# Calcular o aumento de 10% apenas em relação ao preço de fechamento
df['AUMENTO'] = (df['CLOSE'] / df['CLOSE'].shift(1) - 1) > 0.1
# Normalizar as variáveis
df_norm = (df - df.mean()) / df.std()
# Eliminar a primeira linha, que não tem dados de aumento
df_norm.dropna(inplace=True)

# Dividir os dados em conjuntos de treinamento e teste
X = df_norm.drop('AUMENTO', axis=1)
y = df_norm['AUMENTO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)

# Treinar um modelo de regressão logística
lr = LinearRegression()
lr.fit(X_train, y_train)

# Salvar o modelo treinado
joblib.dump(lr, 'modelo_treinado.pkl')

# Avaliar o desempenho do modelo
score = lr.score(X_test, y_test)
print('Acurácia:', score)



Acurácia: 0.03728141057488232


c:\Users\igorb\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1]:
# IMPORTAÇÕES

from consumidor import consumer, atualizacao_carteira
from relatorio import analise_tecnica, grafico, noti_telegram
import yfinance as yf
import finplot as fplt
import numpy as np
import pandas as pd
import mysql.connector
import datetime
import pytz
import os 
from dotenv import load_dotenv, find_dotenv


# CONEXÃO COM BANCO DE DADOS
load_dotenv(find_dotenv())

conn = mysql.connector.connect(
  host= os.getenv('host'),
  user= os.getenv('user'),
  password= os.getenv('password'),
  database="bolsa_valores"
)
cursor = conn.cursor()

# ATUALIZAÇÃO DA BASE DE DADOS DE COTAÇÕES

#consumer()

base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 24 MONTH)', conn)

base


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import joblib
from sklearn.preprocessing import LabelEncoder


# Carregar os dados financeiros de candles a partir de um arquivo CSV
df = base

# Pre-processar os dados
# Eliminar valores ausentes
df.dropna(inplace=True)
# Codificação de categoria
encoder = LabelEncoder()
df.SYMBOL = encoder.fit_transform(df[['SYMBOL']])
# Converter datas em variáveis numéricas
df['DATE'] = pd.to_datetime(df['DATE'].apply(lambda x: datetime.datetime(x.year, x.month, x.day)))
df['DATE'] = df['DATE'].astype('int64') / 10**9
# Calcular o aumento de 10% apenas em relação ao preço de fechamento
df['AUMENTO'] = (df['CLOSE'] / df['CLOSE'].shift(1) - 1) > 0.1
# Normalizar as variáveis
df_norm = (df - df.mean()) / df.std()
# Eliminar a primeira linha, que não tem dados de aumento
df_norm.dropna(inplace=True)

# Dividir os dados em conjuntos de treinamento e teste
X = df_norm.drop('AUMENTO', axis=1)
y = df_norm['AUMENTO']
# Normalizar os dados
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# Definir janela de tempo para previsão
window_size = 30

# Criar função para gerar dados de treinamento e teste
def create_train_test_data(data, window_size):
    X = []
    y = []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, :])
        y.append(data[i, 0])
    X = np.array(X)
    y = np.array(y)
    return X, y

# Gerar dados de treinamento e teste
data = df_norm.to_numpy()
X, y = create_train_test_data(data, window_size)
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Criar modelo de regressão com RNN LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(window_size, data.shape[1])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Treinar modelo
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Salvar o modelo treinado
joblib.dump(model, 'modelo_treinado.pkl')

# Fazer previsões em dados de teste
y_pred = model.predict(X_test)

# Avaliar desempenho do modelo
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R2 Score:", r2)



C:\Users\igorb\AppData\Local\Temp\ipykernel_11860\1532527565.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 24 MONTH)', conn)
c:\Users\igorb\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/50
1209/1209 [==============================] - 20s 14ms/step - loss: 0.0053
Epoch 2/50
1209/1209 [==============================] - 18s 14ms/step - loss: 0.0023
Epoch 3/50
1209/1209 [==============================] - 17s 14ms/step - loss: 0.0022
Epoch 4/50
1209/1209 [==============================] - 17s 14ms/step - loss: 0.0021
Epoch 5/50
1209/1209 [==============================] - 17s 14ms/step - loss: 0.0021
Epoch 6/50
1209/1209 [==============================] - 18s 15ms/step - loss: 0.0020
Epoch 7/50
1209/1209 [==============================] - 21s 17ms/step - loss: 0.0020
Epoch 8/50
1209/1209 [==============================] - 21s 17ms/step - loss: 0.0020
Epoch 9/50
1209/1209 [==============================] - 21s 17ms/step - loss: 0.0020
Epoch 10/50
1209/1209 [==============================] - 20s 17ms/step - loss: 0.0020
Epoch 11/50
1209/1209 [==============================] - 20s 16ms/step - loss: 0.0020
Epoch 12/50
1209/1209 [==============================] - 20s 16

In [11]:
df_norm.sample(5)


,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,DIVIDENDS,AUMENTO
4128,0.360610,0.653061,0.239881,0.239472,0.233368,0.226877,0.020881,0.0,0.0
17785,0.282940,0.561224,0.189028,0.188519,0.193736,0.188249,0.005771,0.0,0.0
25050,0.294036,0.836735,0.356183,0.353669,0.349500,0.343423,0.003361,0.0,0.0
2657,0.381415,0.938776,0.756518,0.755323,0.779208,0.759472,0.053035,0.0,0.0
39887,0.825243,0.683673,0.243932,0.247160,0.248603,0.241873,0.101369,0.0,0.0


In [ ]:
#utilizar modelo treinado


# Carregar o modelo treinado
lr = joblib.load('modelo_treinado.pkl')


# Pre-processar os dados
# Converter datas em variáveis numéricas
novos_dados['data'] = pd.to_datetime(novos_dados['data']).astype(int) / 10**9


# Prever em quantos dias o preço da ação atingirá um aumento de pelo menos 10%
novos_dados = pd.read_csv('novos_dados_candles.csv')
novos_dados_norm = (novos_dados - df.mean()) / df.std()
proba = lr.predict_proba(novos_dados_norm)
idx = (proba[:, 1] > 0.5).argmax()
dias = idx + 1
print('O preço da ação atingirá um aumento de pelo menos 10% em', dias, 'dias.')


